*Prepared for the course "TDPS22: Data Science Programming" at Jönköping University, Teacher: [Marcel Bollmann](mailto:marcel.bollmann@ju.se)*

# Exercise 7: Data Transformation with dplyr & lubridate

This notebook contains exercises on data transformation. We're following [Chapter 5 in _R for Data Science_](https://r4ds.had.co.nz/transform.html), but also jump ahead a bit and include [Chapter 16 on "dates and times"](https://r4ds.had.co.nz/dates-and-times.html).

Concretely, we'll look at the five core functions for data manipulation with dplyr – `filter()`, `arrange()`, `select()`, `mutate()`, and `summarise()` –, data grouping via `group_by()`, how to use pipes to chain function calls via `%>%`, and how to work with dates, date-times, and times-of-day.

### Learning Goals

- Know how to _perform data manipulation_ with dplyr.
- Understand how to _use pipes_.
- Understand how to _work with dates and date-times_ in lubridate, and _time-of-day_ in hms.

### Useful Resources

+ ["Data transformation" in _R for Data Science_](https://r4ds.had.co.nz/transform.html)
+ ["Dates and times" in _R for Data Science_](https://r4ds.had.co.nz/dates-and-times.html)
+ [RStudio Cheatsheets](https://www.rstudio.com/resources/cheatsheets/)
+ [Hands-On Programming with R](https://rstudio-education.github.io/hopr/) _(as a reference)_

In [12]:
library(tidyverse)
library(lubridate)  # this is part of Tidyverse, but needs to be loaded explicitly
library(hms)        # this is part of Tidyverse, but needs to be loaded explicitly
library(dplyr)

- - - 

Let's try loading the **Coffee Chain dataset** that we already saw in Exercise 1:

In [13]:
coffee <- read_csv("data/coffee-chain.csv")

Rows: 10000 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): Ddate, Market, Product, Product Type
dbl (3): Sales, Profit, Expenses

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


The column types are determined automatically, but they are not ideal — the numeric columns are assigned `dbl` (floating-point numbers) while they are actually only ever integer-valued, and the "Market", "Product", "Product Type" columns are good examples for **factors**, since they are categorical variables with a fixed set of possible values. Of course, there's also the question of how to parse the date into a proper `date` variable, but we'll get to that a little later. The other issues are easy to fix, so let's do that immediately:

In [14]:
coffee <- read_csv("data/coffee-chain.csv", col_types="cfffiii")
str(coffee)

spc_tbl_ [10,000 × 7] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Ddate       : chr [1:10000] "9/1/13" "5/1/12" "1/1/13" "8/1/12" ...
 $ Market      : Factor w/ 4 levels "Central","East",..: 1 2 3 3 3 2 1 1 4 1 ...
 $ Product     : Factor w/ 13 levels "Decaf Irish Cream",..: 1 2 3 4 5 6 7 1 2 1 ...
 $ Product Type: Factor w/ 4 levels "Coffee","Espresso",..: 1 2 2 3 1 4 2 1 2 1 ...
 $ Sales       : int [1:10000] 90 203 524 273 201 171 132 225 108 208 ...
 $ Profit      : int [1:10000] 36 56 136 81 -3 26 72 160 37 105 ...
 $ Expenses    : int [1:10000] 37 55 93 68 68 73 31 56 42 43 ...
 - attr(*, "spec")=
  .. cols(
  ..   Ddate = col_character(),
  ..   Market = col_factor(levels = NULL, ordered = FALSE, include_na = FALSE),
  ..   Product = col_factor(levels = NULL, ordered = FALSE, include_na = FALSE),
  ..   `Product Type` = col_factor(levels = NULL, ordered = FALSE, include_na = FALSE),
  ..   Sales = col_integer(),
  ..   Profit = col_integer(),
  ..   Expenses = col_integer()
  .. 

Look at the help for `read_csv()` if you want to know more about the `col_types` argument ...

- - - 

## Filtering, arranging, selecting, mutating

We first look at using four core dplyr functions individually:
- `filter()` is a way to select certain _rows_ of a dataset;
- `arrange()` is a way to sort them;
- `select()` is a way to select certain _columns_ of a dataset; and
- `mutate()` is a way to create new columns from old ones.

**1. Select all rows with products of type "Coffee"!**  _(You should end up with 3383 rows. Look at the first line of the output to see the number of rows!)_

In [30]:
# YOUR CODE HERE
# coffee %>%
filter(coffee, `Product Type` == "Coffee")

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
9/1/13,Central,Decaf Irish Cream,Coffee,90,36,37
12/1/13,West,Amaretto,Coffee,201,-3,68
2/1/13,Central,Decaf Irish Cream,Coffee,225,160,56
9/1/13,Central,Decaf Irish Cream,Coffee,208,105,43
1/1/12,Central,Decaf Irish Cream,Coffee,316,163,64
1/1/12,South,Decaf Irish Cream,Coffee,112,66,24
2/1/13,South,Decaf Irish Cream,Coffee,130,68,43
1/1/12,East,Decaf Irish Cream,Coffee,252,87,81
7/1/13,Central,Decaf Irish Cream,Coffee,70,24,22


**2. Select all rows with negative profit values!** _(You should end up with 686 rows.)_

In [32]:
# YOUR CODE HERE
coffee[coffee$Profit < 0, ]

Ddate,Market,Product,Product Type,Sales,Profit,Expenses
<chr>,<fct>,<fct>,<fct>,<int>,<int>,<int>
12/1/13,West,Amaretto,Coffee,201,-3,68
9/1/12,East,Lemon,Herbal Tea,84,-8,72
4/1/12,East,Decaf Espresso,Espresso,54,-4,26
9/1/12,South,Lemon,Herbal Tea,145,-4,54
9/1/12,South,Lemon,Herbal Tea,89,-5,37
7/1/12,West,Decaf Irish Cream,Coffee,127,-2,71
5/1/13,East,Decaf Irish Cream,Coffee,185,-56,49
5/1/13,West,Mint,Herbal Tea,109,-7,71
9/1/12,South,Decaf Irish Cream,Coffee,77,-16,72


**3. Select all rows with product type "Coffee" _or_ "Espresso" that also come from the "West" market!** _(You should end up with 1712 rows.)_

In [ ]:
# YOUR CODE HERE

**4. Select all rows with "Darjeeling" tea where the _sales_ are greater than 500!** _(You should end up with 76 rows.)_

In [ ]:
# YOUR CODE HERE

**5. Arrange the coffee dataset by "Sales" in descending order!** What product had the highest number of sales in the dataset?

In [ ]:
# YOUR CODE HERE

**6. Select the "Product" and "Product Type" columns and assign them to a variable `products`!**

In [ ]:
# YOUR CODE HERE

Afterwards, you should be able to run the following line of code to get all 13 unique "Product"–"Product Type" combinations:

In [ ]:
unique(products)

**7. Find out what the `any_of()` function does, and how to use it to select all columns whose names are included in the `cols` vector below.**

In [ ]:
cols <- c("Sales", "Inventory", "Budget", "Profit", "Expenses")

In [ ]:
# YOUR CODE HERE

**8. Rearrange the columns so that the "Product Type" column comes first.**

In [ ]:
# YOUR CODE HERE

**9. Rename the "Product Type" column to "Product_Type"!** Notice the underscore. If we change the space to an underscore, we don't have to wrap this column name in backticks all the time. Assign the result to the `coffee` variable again so the change persists!

In [ ]:
# YOUR CODE HERE

_Bonus:_ The `rename_with()` function changes column names based on a function. For example, we can change them all to lowercase via:

In [ ]:
coffee <- rename_with(coffee, tolower)

I'll refer to column names in lowercase from here on.

**10. Create two new columns: a column "revenue" that is the _sum_ of "profit" and "expenses"; and a column "margin" that is "profit" _divided by_ "revenue".** Try to create both columns in a single `mutate()` statement.

In [ ]:
# YOUR CODE HERE

**11. Make a new column "profit_above_avg" that is `TRUE` when the "profit" is above the average of the dataset, and `FALSE` otherwise.** _Note:_ You can get the average by calling `mean()`.

In [ ]:
# YOUR CODE HERE

- - - 

## Pipes, grouping, and summarising

_Grouping_ and _summarising_ data is mainly done via `group_by()` and `summarise()`. They are often used together, and as such, it's a good idea to take a look at _pipes_ first before we get more into them.

**12. Rewrite the cell below to a version _without pipes_ and _only one function per line_.** Use intermediate variables to store results of function calls. Basically, make sure that you understand what exactly is happening in the pipe.

In [ ]:
coffee %>%  
  pull(profit) %>%
  sum == 643034

In [ ]:
# YOUR CODE HERE

**13. Rewrite the cell below to a version _with pipes_.** You can use the `pull()` function to "pull out" specific columns of the dataset, or use the dot `.` in a pipe as a placeholder for the input variable.

In [ ]:
tmp <- filter(coffee, sales > 100)
tmp <- count(tmp)
tmp$n

In [ ]:
# YOUR CODE HERE

**14. How many instances of each product type are there, and which one generates the most/the least profit, on average?** You'll need to chain a `group_by()` and a `summarise()` call to solve this; try to do this with a pipe (`%>%`) instead of intermediate variables, if possible. You should end up with a tibble that has four rows, one for each product type, and columns for the _name_, _count_, and _mean profit_ of each product type.

In [ ]:
# YOUR CODE HERE

**15. What's the _minimum, maximum, average, and median profit_ per "product"?** The resulting tibble should have unique "product" values (e.g., Amaretto, Caffe Latte, ...) as _rows_, and the statistics about the distribution of their "profit" values as _columns_.

In [ ]:
# YOUR CODE HERE

**16. Filter out all products where we don't have data from all markets.** This is potentially a bit trickier. You might want to start by finding an expression that gives you the number of unique markets (you've seen an example for getting unique values at the top of this notebook, and you can look at the `length()` function to obtain a count), then try to figure out how to use grouping and filtering to achieve the desired result.

_Note:_ Like in Exercise 1, where we did this before in Python, an indication that you got the right expression is that you should end up with a filtered dataset that has 7426 rows.

In [ ]:
# YOUR CODE HERE

- - - 

## Dates and date-times

In this final part, we combine what we've practised so far with `lubridate`, which gives us functionality to work with dates and date-times. Here, [Chapter 16](https://r4ds.had.co.nz/dates-and-times.html) of the book comes into play.

**17. Find the right `lubridate` function to convert the following string to a `date` object!** This string is in the same format as the "Ddate" column of our coffee dataset, so knowing the conversion function will help us work with it.

In [ ]:
ddate1 <- "8/21/13"  # a.k.a. 21st August, 2013

In [ ]:
# YOUR CODE HERE

**18. Modify the `coffee` dataset so that the "ddate" column contains a parsed "date" object instead of a string!** In the transformed dataset, "ddate" should now be a column of type `<date>` instead of `<chr>`.

In [ ]:
# YOUR CODE HERE

**19. Select all _rows_ of the dataset where the _month_ is August.** Use your transformed dataset from the previous question.

In [ ]:
# YOUR CODE HERE

**20. How many "ddate"s in the dataset fell on a Sunday, and how many rows do we have for these dates?** I'll give you the answer in table form below – produce a function pipeline that ultimately gives the same result:

|   ddate    | count | wday   |
|------------|-------|--------|
| 2012-01-01 | 565   | Sunday |
| 2012-04-01 | 434   | Sunday |
| 2012-07-01 | 431   | Sunday |
| 2013-09-01 | 410   | Sunday |
| 2013-12-01 | 99    | Sunday |

In [ ]:
# YOUR CODE HERE

The "coffee chain" dataset doesn't contain any times or date-times, so we'll try to answer some "artificial" questions that deal with those for now. There will be more opportunities to practice working with times and date-times in the assignment (and potentially other exercises).

**21. How many _seconds_ passed between the birth of _Queen Elizabeth II of the United Kingdom_ and _King Carl XVI Gustaf of Sweden_?** Wikipedia is very specific about the times that royals were born. Concretely, you can find that:

- Queen Elizabeth II was born on April 21, 1926, at 2:40 a.m. GMT in London.
- Carl XVI Gustaf was born on April 30, 1946, at 10:20 a.m. in Stockholm.

Create date-time objects for the birth of these two monarchs (don't forget about time zones!) and compute the difference _in seconds_ between them!

In [ ]:
# YOUR CODE HERE

**22. How many seconds after 10:00:00 is it right now?** This is a use case for the `hms` package, which isn't explicitly described in the book, but is simple & useful enough that we can just take a brief look at its documentation. Check `?hms` or [the "hms" website](https://hms.tidyverse.org/) for a brief overview of the package, and use the `hms()`, `as_hms()`, and `now()` functions to answer the question.

In [ ]:
# YOUR CODE HERE